#  Calibration Image Master Creation notebook

This notebook calibrates Bias images, Dark images and Flat images and combines them into their respective Master files

Made by: Harlan Shaw <harlan.shaw@ucalgary.ca>

## Required Python packages:
[Astropy](https://www.astropy.org/)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/index.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)

It's recommended you download and install [Anaconda](https://www.anaconda.com/products/individual#Downloads) as this contains a Python environment and Astropy.

You will need to install CCDProc using the Conda install command: `conda install -c conda-forge ccdproc`

This will also install Astropy and Astro-SCRAPPY if not already installed.

## Citations:
With the projects used above, any use of this notebook requires the citations of the Python packages and papers from which they are derived. 
Their citations are located here:

[Astropy](https://www.astropy.org/acknowledging.html)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/citation.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)



## How to Use

It's recommended you click "Run All Cells" to run this notebook, but you may optionally run each cell individually.

This notebook requires the location of your raw bias images as either a full location or as a relative path if you have the bias images in the folder that this notebook is located in.

Example:

Full location: C:\path\to\bias

Relative: path\to\bias



# Setup

All variables are set up in this cell. If this cell is not run, the program will not run.

In [3]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.stats import mad_std
from astropy.units import adu, second
from astropy.wcs import FITSFixedWarning
from ccdproc import ImageFileCollection, combine, subtract_bias, subtract_dark, cosmicray_lacosmic
import matplotlib.pyplot as plt
import numpy as np
import logging
import warnings

warnings.filterwarnings("ignore", category=FITSFixedWarning)


GIGABYTE = 1e9 # 1 billion bytes
MEMORY_LIMIT = 10*GIGABYTE

data_directory = Path(".") # directory that notebooks are in.
DARKS_PATH = data_directory / "Darks"
BIAS_PATH = data_directory / "Bias"
FLATS_PATH = data_directory / "Flats"
reduced_path = data_directory / "reduced"
reduced_path.mkdir(exist_ok=True)
#useful functions
def inv_average(data):
    return 1/np.average(data)

def inv_median(data):
    return 1/np.median(data)


# Stage 1: Bias

During this stage the master Bias is made, it is integral to the rest of the process and must be done first.

In [2]:
biases = ImageFileCollection(BIAS_PATH, glob_include="*bias*.fit*")
paths = []
for bias in biases.files:
    bias_path = BIAS_PATH / bias
    paths.append(bias_path)
combined_bias = combine(paths, 
                        method="median", 
                        mem_limit = MEMORY_LIMIT,
                        sigma_clip=True,
                        sigma_clip_low_thresh=5,
                        sigma_clip_high_thresh=5,
                        sigma_clip_func=np.median,
                        sigma_clip_dev_func=mad_std,
                        unit=adu)
combined_bias.meta["IMAGETYP"] = "Bias"
combined_bias.meta['HISTORY'] = f"Combined {len(biases.files)} images by median"
combined_bias = combined_bias.to_hdu()
combined_bias[0].scale("int32")
combined_bias[0].writeto(reduced_path / "combined_bias.fit")

NameError: name 'ImageFileCollection' is not defined


# Stage 2: Dark

Here the master Dark is made. This process requires the use of the master Bias, so Stage 1 must be run first

In [8]:
darks = ImageFileCollection(DARKS_PATH, glob_include="*.fit*")
logging.info(f"Found {len(darks.files)} dark files")

paths = []
for dark in darks.files:
    dark_path = DARKS_PATH / dark
    paths.append(dark_path)
combined_dark = combine(paths, 
                        method="median", 
                        mem_limit = MEMORY_LIMIT,
                        sigma_clip=True,
                        sigma_clip_low_thresh=5,
                        sigma_clip_high_thresh=5,
                        sigma_clip_func=np.median,
                        sigma_clip_dev_func=mad_std,
                        unit=adu)

combined_bias_path = reduced_path / "combined_bias.fit"
combined_bias = CCDData.read(combined_bias_path, unit=adu)

combined_dark = subtract_bias(combined_dark, combined_bias, add_keyword={"HISTORY": f"Subtracted master bias {combined_bias_path.name}"})
combined_dark.meta["IMAGETYP"] = "Dark"
combined_dark.meta['combined'] = True
combined_dark.meta['HISTORY'] = f"Combined {len(darks.files)} images by median"
combined_dark = combined_dark.to_hdu(hdu_uncertainty=None)
combined_dark[0].scale("int32")
combined_dark[0].writeto(reduced_path / "combined_dark.fit")

INFO:astropy:splitting each image into 6 chunks to limit memory usage to 10000000000.0 bytes.


INFO: splitting each image into 6 chunks to limit memory usage to 10000000000.0 bytes. [ccdproc.combiner]


C:\Users\harla\anaconda3\envs\install\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]



# Stage 3: Flat

Here the master Flat is made. This process requires the use of the master Bias and master Dark, so Stage 1 and 2 must be run first

### Part 1: Calibrate the flat files

In [4]:
flats = ImageFileCollection(FLATS_PATH, glob_include="*")
INTERMEDIARY_PATH = Path(FLATS_PATH / "calibrated")
INTERMEDIARY_PATH.mkdir(exist_ok=True)

for flat in flats.files:
    flat_name = Path(flat).stem 
    modified_flat_name = INTERMEDIARY_PATH / f"{flat_name}_b_d.fit"
    if not modified_flat_name.is_file():
        flat_ccd = CCDData.read(FLATS_PATH / flat, unit=adu)
        try: 
            flat_ccd = subtract_bias(ccd=flat_ccd, 
                                    master=combined_bias, 
                                    add_keyword={"HISTORY": f"Subtracted master bias combined_bias.fit"})
        except:
            combined_bias = reduced_path.joinpath("combined_bias.fit")
            combined_bias = CCDData.read(combined_bias)
            flat_ccd = subtract_bias(ccd=flat_ccd, 
                                    master=combined_bias, 
                                    add_keyword={"HISTORY": f"Subtracted master bias combined_bias.fit"})
        try: 
            flat_ccd = subtract_dark(ccd=flat_ccd, 
                                    master=combined_dark, 
                                    exposure_time="EXPTIME", 
                                    exposure_unit=second, 
                                    scale=True,
                                    add_keyword={"HISTORY": f"Subtracted master bias combined_dark.fit"})
        except:
            combined_dark = reduced_path.joinpath("combined_dark.fit")
            combined_dark = CCDData.read(combined_dark)
            flat_ccd = subtract_dark(ccd=flat_ccd, 
                                    master=combined_dark, 
                                    exposure_time="EXPTIME", 
                                    exposure_unit=second, 
                                    scale=True,
                                    add_keyword={"HISTORY": f"Subtracted master bias combined_dark.fit"})
        #flat_ccd = cosmicray_lacosmic(ccd=flat_ccd, gain=1.42, readnoise=15.1, fsmode="convolve", cleantype="median", gain_apply=True)
        ccd_exposure = float(flat_ccd.header["exptime"])
        flat_ccd = flat_ccd.to_hdu(hdu_uncertainty=None)
        flat_ccd[0].writeto(modified_flat_name)

### Part 2: Combine the flat files

In [5]:

INTERMEDIARY_PATH = Path(FLATS_PATH / "calibrated") 
calibrated_flats = ImageFileCollection(INTERMEDIARY_PATH, glob_include="*.fit*")
paths = []
for flat in calibrated_flats.files:
    flat_path = INTERMEDIARY_PATH / flat
    paths.append(flat_path)
    
combined_flat = combine(paths, 
                        method="median", 
                        mem_limit = MEMORY_LIMIT,
                        scale= inv_median,
                        sigma_clip=True,
                        sigma_clip_low_thresh=5,
                        sigma_clip_high_thresh=5,
                        sigma_clip_func=np.median,
                        sigma_clip_dev_func=mad_std,
                        dtype="float32",
                        unit=adu)
combined_flat.meta["IMAGETYP"] = "Flat"
combined_flat.meta['combined'] = True
combined_flat.meta['HISTORY'] = f"Combined {len(paths)} images by mean"

combined_flat = combined_flat.to_hdu(hdu_uncertainty=None)
combined_flat[0].writeto(reduced_path / "combined_flat.fit")

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:splitting each image into 4 chunks to limit memory usage to 10000000000.0 bytes.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: splitting each image into 4 chunks to limit memory usage to 10000000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
